# Face Detection Challenge

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks

### Objectives
-  visualize images
-  preprocess images for Neural Networks
-  fit a custom CNN for a regression task
-  fine-tune a analyse model performance

## 1. Visualize images

👉 Load the dataset (200 Mo)

In [2]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/images.csv")

KeyboardInterrupt: 

Each row represent the image of a face in black and white
- `age` is the age in year
- `pixels` contains the 2304 (= 48 * 48) flatten values of each pixel in the image, in a black scale from 0 to 255, stored as string

❓ Plot the histogram of age in your dataset

In [ ]:
### TODO
sns.histplot(data.age);

❓ Visually display one image of your choice.

In [ ]:
### TODO
img_1 = np.fromstring(data.pixels[0],sep=' ').astype(int)
img_1 = img_1.reshape(48, 48)
plt.imshow(img_1, cmap=plt.cm.gray)

❓ What is the physical characteristic of the image at **index** `10000`?

In [ ]:
img_10000 = np.fromstring(data.pixels[10000],sep=' ').astype(int)
img_10000 = img_10000.reshape(48, 48)
plt.imshow(img_10000, cmap=plt.cm.gray)

In [ ]:
# Store your result in the variable below

gender = "male"
# gender = "female"

# smile = True
smile = False

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('C14',
                         gender=gender,
                         smile=smile)
result.write()

## 2. Processing

❓ Your goal is to train a convolutional neural network model to predict the age of a person based solely on a picture of his/her face.
- Create your feature matrix `X` as nparray of shape suited for a CNN, scaled between 0 and 1
- Create your target vector `y`
- Create a holdout set (`X_train`, `y_train`) (`X_test`, `y_test`) keeping `30%` in the test set, randomly sampled out of the whole dataset

😌 Don't worry, you will be given the solution in the next question in case you don't make it

In [ ]:
## YOUR CODE
def transform(image):
    img = np.fromstring(image, dtype=int, sep=' ').reshape(48, 48)
    scaler = MinMaxScaler()
    return scaler.fit_transform(img)
X = pd.Series(list(map(transform, data.pixels)))
y = pd.Series(list(data.age))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('C1415', 
                         X_train_shape = X_train.shape,
                         y_train_shape = y_train.shape,
                         first_image = X_train[0]
                        )
result.write()

## Convolutional Neural Network for Age prediction

❓ Build a convolutional neural network

- Do not use transfer learning
- Store number of trainable parameters in a variable `params_number`
- Plot your metric & loss at each epoch
- make sure your model does not overfit with appropriate control techniques
- Compute the mean absolute error on your test set and store it as `mae_test`
- Compute, store and compare it with a `mae_baseline`

💡 You will not be judged by the computing power of your computer, but you should obtain significantly better performance than the baseline in less than 3 minutes, even without GPUs

👉 Feel free to start back from the solution by running the cell below (~500Mo download)

In [ ]:
import pickle

file_names = ["images_X_train.pickle", "images_X_test.pickle", "images_y_train.pickle", "images_y_test.pickle"]
pickles = dict()

for file_name in file_names:
    url = f"https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/{file_name}"
    command = f"curl --output {file_name} {url}"
    ! eval {command}
    with open(file_name, 'rb') as handle:
        pickles[file_name] = pickle.load(handle)
    command = f"rm {file_name}"
    ! eval {command}
        
X_train, X_test, y_train, y_test = pickles.values()

In [ ]:
### YOUR CODE

In [ ]:
X_train.shape

In [ ]:
model = models.Sequential()

# input
model.add(layers.Conv2D(20, (4,4), input_shape=(48, 48, 1)))
model.add(layers.MaxPool2D(pool_size=(2,2))) 

# layers
model.add(layers.Conv2D(32, (3,3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Flatten())

#output
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

model.compile(loss='mse',
              optimizer='adam',
              metrics=['mae'])

In [ ]:
es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train,
          batch_size=16,
          epochs=10,
          validation_split=0.3,
          callbacks=[es])

history = model.history

In [ ]:
# number of trainable parameters
params_number = model.count_params()

In [ ]:
# baseline
mae_baseline = pd.Series(y_test).apply(lambda x : abs(data.age.mean() - x)).mean()
mae_baseline

In [ ]:
# mae test
mae_test = model.evaluate(X_test, y_test, verbose=1)[1]
mae_test

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mae')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult('C1516',
                         params_number=params_number,
                         mae_baseline = mae_baseline,
                         mae_test = mae_test,
                        )
result.write()